In [1]:
import numpy as np
from langchain_community.llms import Tongyi
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Tongyi(streaming=True, callbacks=[StreamingStdOutCallbackHandler()],temperature=0.7)

In [2]:
prompt0="请介绍一下微观经济学的前提基础"
llm.invoke(prompt0)

微观经济学是研究个体经济单位，如家庭、企业等在市场中的行为和决策的学科。它的前提基础主要包括：

1. **理性人假设**：微观经济学假定每个经济主体都是理性的，他们会根据自身利益最大化来做出决策。这意味着人们会考虑成本和收益，以达到最佳的经济效果。

2. **资源稀缺性假设**：微观经济学认为资源是有限的，但人类的需求是无限的。因此，必须做出选择以最优的方式分配资源。

3. **市场机制**：市场经济通过价格机制来调节供求关系，价格的变化可以引导消费者和生产者的决策，使资源得到有效的配置。

4. **边际效用递减原理**：这是消费者行为理论的基础，它指出在一定条件下，随着消费量的增加，商品或服务对消费者的边际效用（即每多消费一单位带来的额外满足感）会逐渐减少。

5. **完全信息假设**：在理想情况下，微观经济学假设所有市场参与者都拥有完全且准确的信息，以便做出最佳决策。然而，在现实世界中，这个假设往往不成立。
这些基本前提为微观经济学的理论构建和分析提供了基础，并帮助我们理解经济主体如何在市场中互动和决策。

'微观经济学是研究个体经济单位，如家庭、企业等在市场中的行为和决策的学科。它的前提基础主要包括：\n\n1. **理性人假设**：微观经济学假定每个经济主体都是理性的，他们会根据自身利益最大化来做出决策。这意味着人们会考虑成本和收益，以达到最佳的经济效果。\n\n2. **资源稀缺性假设**：微观经济学认为资源是有限的，但人类的需求是无限的。因此，必须做出选择以最优的方式分配资源。\n\n3. **市场机制**：市场经济通过价格机制来调节供求关系，价格的变化可以引导消费者和生产者的决策，使资源得到有效的配置。\n\n4. **边际效用递减原理**：这是消费者行为理论的基础，它指出在一定条件下，随着消费量的增加，商品或服务对消费者的边际效用（即每多消费一单位带来的额外满足感）会逐渐减少。\n\n5. **完全信息假设**：在理想情况下，微观经济学假设所有市场参与者都拥有完全且准确的信息，以便做出最佳决策。然而，在现实世界中，这个假设往往不成立。\n\n这些基本前提为微观经济学的理论构建和分析提供了基础，并帮助我们理解经济主体如何在市场中互动和决策。'

In [3]:
llm.get_num_tokens("fasfdsafreqwrwe") # 需要依赖torch

OSError: Can't load tokenizer for 'gpt2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'gpt2' is the correct path to a directory containing all relevant files for a GPT2TokenizerFast tokenizer.

In [4]:
help(llm)

Help on Tongyi in module langchain_community.llms.tongyi object:

class Tongyi(langchain_core.language_models.llms.BaseLLM)
 |  Tongyi(*, name: Optional[str] = None, cache: ForwardRef('Union[BaseCache, bool, None]') = None, verbose: bool = None, callbacks: ForwardRef('Callbacks') = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, custom_get_token_ids: Optional[Callable[[str], List[int]]] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, client: Any = None, model_name: str = 'qwen-plus', model_kwargs: Dict[str, Any] = None, top_p: float = 0.8, dashscope_api_key: Optional[str] = None, streaming: bool = False, max_retries: int = 10) -> None
 |  
 |  Tongyi Qwen large language models.
 |  
 |  To use, you should have the ``dashscope`` python package installed, and the
 |  environment variable ``DASHSCOPE_API_KEY`` set with your API key, or pass
 |  it as a named parameter to the constructor.
 |  
 |  Example:
 | 

In [5]:
dir(llm)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [6]:
llm_di = Tongyi(temperature=0.2)

In [7]:
s = "请帮我写一副关于龙年的对联"

In [8]:
llm.invoke(s)

上联：龙舞长空，瑞气盈门福满堂
下联：年华似锦，盛世如歌春意浓
横批：龙年大吉，喜结四海祥云

'上联：龙舞长空，瑞气盈门福满堂\n下联：年华似锦，盛世如歌春意浓\n横批：龙年大吉，喜结四海祥云'

In [9]:
llm_di.invoke(s)

'上联：龙舞长空，瑞气盈门喜庆春\n下联：年华似锦，福星高照乐融融\n横批：龙年大吉，盛世繁华'

In [11]:
llm_ga = Tongyi(temperature=0.9)
res = llm_ga.generate(["写一首唐诗绝句","写一首宋词《贺新郎》"])

In [16]:
print(res.generations[0][0].text)
res.generations[0][0].generation_info

春风吹柳绿如丝，桃花绽放映朝辉。
江水潺潺流不息，渔舟唱晚入画帷。


{'finish_reason': 'stop',
 'request_id': '85fd13ca-2936-92aa-b578-317b75cdf056',
 'token_usage': {'input_tokens': 14, 'output_tokens': 29, 'total_tokens': 43}}

In [17]:
print(res.generations[1][0].text)
res.generations[1][0].generation_info

《贺新郎·赋壮志豪情》

举头望明月，豪情盈胸臆。万里江山，英雄何处觅知音，剑光如龙舞。醉里挑灯看剑，梦回吹角连营，壮志凌云深。 

烽火戏诸侯，世事如棋局。谁人挥毫，书尽人间疾苦，笔下生风雷。铁马冰河入梦来，金戈铁马，气吞万里如虎，誓将功名铸。

倚天照海花无数，世间英豪争锋处。笑谈间，樯橹灰飞烟灭，谈笑风生无惧。待到功成名就时，回首往事，一杯浊酒祭英魂，壮志长存，不负韶华年少，贺此生，新郎英勇，词韵犹在耳。


{'finish_reason': 'stop',
 'request_id': 'bd16be83-9b58-9a35-a083-5c7341bbdd62',
 'token_usage': {'input_tokens': 17,
  'output_tokens': 187,
  'total_tokens': 204}}

In [18]:
llm_ga.invoke("这首贺新郎是否符合词牌格律？") # 显然模型调用没法识别历史提问上下文

'很抱歉，由于您没有提供具体的贺新郎词，我无法直接判断是否符合词牌格律。请您提供完整的词，我会帮您分析是否符合贺新郎的词牌格律。贺新郎，又名《金缕曲》《贺新凉》，是宋代刘克庄创制的词牌，双调，一百十六字，上下片各十句六仄韵。'

In [19]:
from langchain import PromptTemplate

template = """
我希望你能充当新公司的命名顾问。
一个生产{product}的公司的好名字是什么？
"""

prompt_template = PromptTemplate(
    input_variables=["product"],
    template=template,
)
prompt = prompt_template.format(product="彩色袜子")

In [20]:
print(llm_ga.invoke(prompt))

为你的彩色袜子公司取名是一项有趣的任务。以下是一些创意的建议：

1. "RainbowSteps"
2. "SoleSplash"
3. "VibrantToes"
4. "ColorCraze Socks"
5. "PiedPiper Socks"
6. "FiestaFeet"
7. "SpectrumSocks"
8. "ChromaticSoles"
9. "ZestyThreads"
10. "JazzyJamboree"

在选择名字时，考虑公司的品牌定位、目标市场和你希望传达的情感。确保名字易于记忆，且在商标搜索中是独特的。祝你的新公司取得成功！


In [21]:
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"], 
    template="给我讲一个{adjective}的关于{content}的笑话。"
)
prompt = multiple_input_prompt.format(adjective="有趣的", content="小鸡")

In [22]:
print(llm_ga.invoke(prompt))

好的，这里有一个关于小鸡的笑话：

为什么小鸡总是这么乐观？

因为它们每天都能看到新的日出（蛋）！


In [2]:
from langchain import PromptTemplate, FewShotPromptTemplate

# 首先，创建Few Shot示例列表
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

# 接下来，我们指定用于格式化示例的模板。
# 我们使用`PromptTemplate`类来实现这个目的。
example_formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt_template = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

# 最后，创建`FewShotPromptTemplate`对象。
few_shot_prompt = FewShotPromptTemplate(
    # 这些是我们要插入到提示中的示例。
    examples=examples,
    # 这是我们在将示例插入到提示中时要使用的格式。
    example_prompt=example_prompt_template,
    # 前缀是出现在提示中示例之前的一些文本。
    # 通常，这包括一些说明。
    prefix="Give the antonym of every input\n",
    # 后缀是出现在提示中示例之后的一些文本。
    # 通常，这是用户输入的地方。
    suffix="Word: {input}\nAntonym: ",
    # 输入变量是整个提示期望的变量。
    input_variables=["input"],
    # 示例分隔符是我们将前缀、示例和后缀连接在一起的字符串。
    example_separator="\n",
)

# 现在，我们可以使用`format`方法生成一个提示。
prompt = few_shot_prompt.format(input="big")
print(prompt)

Give the antonym of every input

Word: happy
Antonym: sad

Word: tall
Antonym: short

Word: big
Antonym: 


In [4]:
llm.invoke(few_shot_prompt.format(input="生机"))

死气 (sǐ qì) - literally means "dead air" or "lack of vitality" in Chinese

'死气 (sǐ qì) - literally means "dead air" or "lack of vitality" in Chinese'

In [8]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage

chat_ty = ChatTongyi(
    streaming=True,
)
res = chat_ty.stream([HumanMessage(content="hi")], streaming=True)
try:
    for chunk in res:
        print(chunk)
except TypeError as e:
    print("")

content='Hello' id='run-9ea803e5-136c-40ab-b9cd-d3d557d85898'
content='!' id='run-9ea803e5-136c-40ab-b9cd-d3d557d85898'
content=' How' id='run-9ea803e5-136c-40ab-b9cd-d3d557d85898'
content=' can I assist you today' id='run-9ea803e5-136c-40ab-b9cd-d3d557d85898'
content='?' response_metadata={'finish_reason': 'stop', 'request_id': 'd3ccaf75-7658-9e0a-b981-c343e0314321', 'token_usage': {'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18}} id='run-9ea803e5-136c-40ab-b9cd-d3d557d85898'


In [12]:
res = chat_ty.stream([HumanMessage(content="帮我写副对联")], streaming=True)

In [13]:
try:
    for chunk in res:
        print(chunk.content)
except TypeError as e:
    print("")

当然
可以
，
这里有一副适合春节
的对联供您参考：

上
联：红梅傲雪春来到
，富贵吉祥喜盈门。
下
联：绿竹报岁福无
边，平安如意乐满堂。

横批：新春纳福

如果您
有其他主题或场合的需求，也
请告诉我，我会尽力为您定制。


In [15]:
from langchain.chains import LLMChain
 
prompt = PromptTemplate(
    input_variables=["product"],
    template="我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？",
)

In [19]:
chain = prompt | llm

In [17]:
chain.invoke("彩色袜子")

1. 彩韵袜业：这个名字融合了"彩"（彩色）和"韵"（韵律，寓意袜子的多样性和时尚感），体现出产品的特色。

2. 明彩足下：明彩代表明亮的色彩，足下则直接指代袜子，寓意穿上公司的彩色袜子能让人的步伐更加生动。

3. 色舞袜坊：色舞寓意色彩的灵动与活力，坊则给人一种亲切和手工艺的感觉，适合小型或有独特工艺的袜子公司。

4. 七彩虹袜：七彩虹代表了丰富的颜色，直接关联到公司的主要产品，同时也传达出快乐、积极的品牌形象。

5. 魅彩袜语：魅彩表示袜子的吸引力和色彩魅力，"袜语"则暗示了袜子可以作为表达个性的一种方式，富有创意。

'1. 彩韵袜业：这个名字融合了"彩"（彩色）和"韵"（韵律，寓意袜子的多样性和时尚感），体现出产品的特色。\n\n2. 明彩足下：明彩代表明亮的色彩，足下则直接指代袜子，寓意穿上公司的彩色袜子能让人的步伐更加生动。\n\n3. 色舞袜坊：色舞寓意色彩的灵动与活力，坊则给人一种亲切和手工艺的感觉，适合小型或有独特工艺的袜子公司。\n\n4. 七彩虹袜：七彩虹代表了丰富的颜色，直接关联到公司的主要产品，同时也传达出快乐、积极的品牌形象。\n\n5. 魅彩袜语：魅彩表示袜子的吸引力和色彩魅力，"袜语"则暗示了袜子可以作为表达个性的一种方式，富有创意。'

In [21]:
input_list = [
    {"product": "袜子"},
    {"product": "电脑"},
    {"product": "男鞋"}
]
chain.invoke(input_list)

当然可以，以下是我为这个多元产品公司提出的五个中文名称建议：
1. 跨悦科技（跨越生活，悦享品质，同时“跨悦”也能暗示产品的多样性）
2. 着装未来（"着装"涵盖了衣物类，"未来"则与电脑的高科技感相呼应）
3. 通途舒步（"通途"象征电脑带来的信息通路，"舒步"则关乎鞋类产品，整体寓意舒适前行）
4. 多元智足（"多元"代表产品种类丰富，"智"对应电脑，"足"对应鞋子，也寓意智慧生活）
5. 缘步云袜（"缘步"有随缘而行之意，"云"则可关联到电脑的云计算，"袜"直接指代该产品线）

请根据公司的品牌定位、目标市场和喜好来选择最适合的名字。

'当然可以，以下是我为这个多元产品公司提出的五个中文名称建议：\n\n1. 跨悦科技（跨越生活，悦享品质，同时“跨悦”也能暗示产品的多样性）\n2. 着装未来（"着装"涵盖了衣物类，"未来"则与电脑的高科技感相呼应）\n3. 通途舒步（"通途"象征电脑带来的信息通路，"舒步"则关乎鞋类产品，整体寓意舒适前行）\n4. 多元智足（"多元"代表产品种类丰富，"智"对应电脑，"足"对应鞋子，也寓意智慧生活）\n5. 缘步云袜（"缘步"有随缘而行之意，"云"则可关联到电脑的云计算，"袜"直接指代该产品线）\n\n请根据公司的品牌定位、目标市场和喜好来选择最适合的名字。'

In [22]:
for input in input_list:
    chain.invoke(input)

当然可以，以下是我为您的袜子公司提出的五个中文名字建议：
1. "舒步袜业"：这个名字强调了袜子的舒适性和适合行走的特点，同时也表达了轻松愉快的步履之意。
2. "纤丝韵"：这个名字突出了袜子材质的细腻和优雅，韵字则带出了一种时尚和节奏感。
3. "足下温馨"：简洁易记，寓意袜子带来的温暖和关怀，让人感到亲切。
4. "行云袜"：借用“行云流水”的成语，寓意穿着者的步伐轻盈自在，同时展现了袜子的高品质。
5. "彩织生活"：强调袜子的多彩选择和它为日常生活增添的色彩，也暗示了公司产品的丰富多样。

希望这些建议能对您有所帮助！当然可以，以下是我为您的电脑生产公司提出的五个可能的中文名字建议：

1. "智启科技"：这个名字强调了智慧和创新的启动力量，代表电脑技术的智能开启未来。
2. "灵翼计算机"："灵翼"寓意灵活、快速，象征电脑的高效性能和无限可能。
3. "迅捷芯动"：强调电脑的运行速度和处理器的强劲动力，同时也表达了动态和活力。
4. "明略电子"："明略"意味着明智的策略和清晰的视野，暗示公司的前瞻性和产品的高质量。
5. "睿智未来"：突出了电脑作为智慧工具的角色，以及对未来智能生活的引领。

这些名字都包含了积极的寓意，希望能给您一些灵感。在选择时，还可以考虑与公司的品牌理念、目标市场和产品特性相匹配的因素。当然可以，以下是我为您的男鞋公司提出的五个可能的中文名字建议：

1. 雄步天下：这个名字寓含了男性稳健、自信的步伐，同时也暗示了公司的产品能够帮助消费者走遍天下。
2. 驭风绅士：驭风象征速度和力量，绅士则代表了品质和优雅，适合定位中高端市场的男鞋品牌。
3. 革履先生：以“革”突出鞋子的主要材质，而“先生”则给人一种稳重、有品位的感觉。
4. 峻履前行：峻字寓意坚韧不拔，前行则象征着前进的动力，适合强调产品质量和进取精神的品牌。
5. 稳行者：稳字体现了鞋子的舒适性和稳定性，行者则代表了穿着者的旅行和生活态度。

每个名字都有其独特的含义，希望能帮到您。在选择时，还需要考虑品牌的定位、目标市场以及想要传达的品牌形象。

In [24]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="请列出五个 {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)
prompt

PromptTemplate(input_variables=['subject'], partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'}, template='请列出五个 {subject}.\n{format_instructions}')

In [25]:
input0 = prompt.format(subject="中国的名山")
output = llm.invoke(input0)
res0=output_parser.parse(output)
res0

泰山, 黄山, 衡山, 华山, 嵩山

['泰山', '黄山', '衡山', '华山', '嵩山']

### 其他解析器

In [ ]:
from langchain.output_parsers import DatetimeOutputParser,StructuredOutputParser
# 日期时间解析器:DatetimeOutputParser
# 结构化输出解析器:StructuredOutputParser

### 历史消息

## Memory

大多数的 LLM 应用程序都会有一个会话接口，允许我们和 LLM 进行多轮的对话，并有一定的上下文记忆能力。但实际上，模型本身是不会记忆任何上下文的，只能依靠用户本身的输入去产生输出。而实现这个记忆功能，就需要额外的模块去保存我们和模型对话的上下文信息，然后在下一次请求时，把所有的历史信息都输入给模型，让模型输出最终结果。

而在 LangChain 中，提供这个功能的模块就称为 Memory，用于存储用户和模型交互的历史信息。在 LangChain 中根据功能和返回值的不同，会有多种不同的 Memory 类型，主要可以分为以下几个类别：

* 对话缓冲区内存（ConversationBufferMemory），最基础的内存模块，用于存储历史的信息
* 对话缓冲器窗口内存（ConversationBufferWindowMemory），只保存最后的 K 轮对话的信息，因此这种内存空间使用会相对较少
* 对话摘要内存（ConversationSummaryMemory），这种模式会对历史的所有信息进行抽取，生成摘要信息，然后将摘要信息作为历史信息进行保存。
* 对话摘要缓存内存（ConversationSummaryBufferMemory），这个和上面的作用基本一致，但是有最大 token 数的限制，达到这个最大 token 数的时候就会进行合并历史信息生成摘要

值得注意的是，对话摘要内存的设计出发点就是语言模型能支持的上下文长度是有限的（一般是 2048），超过了这个长度的数据天然的就被截断了。这个类会根据对话的轮次进行合并，默认值是 2，也就是每 2 轮就开启一次调用 LLM 去合并历史信息。

In [33]:
# 链外管理内存
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("hi!")
history.add_ai_message("whats up?")
history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

In [30]:
# 对话缓冲区内存（ConversationBufferMemory）
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:
Hello! How can I assist you today? I'm here to provide information, answer questions, or engage in some interesting chatter. Feel free to ask me anything!
> Finished chain.


"Hello! How can I assist you today? I'm here to provide information, answer questions, or engage in some interesting chatter. Feel free to ask me anything!"

In [31]:
conversation.predict(input="你是谁")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? I'm here to provide information, answer questions, or engage in some interesting chatter. Feel free to ask me anything!
Human: 你是谁
AI:
我是一个人工智能助手，被设计来帮助用户解答问题、提供信息或进行友好的对话。你可以把我想象成一个知识丰富的聊天机器人，随时准备与你交流。
> Finished chain.


'我是一个人工智能助手，被设计来帮助用户解答问题、提供信息或进行友好的对话。你可以把我想象成一个知识丰富的聊天机器人，随时准备与你交流。'

In [32]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Hi there!'),
 AIMessage(content="Hello! How can I assist you today? I'm here to provide information, answer questions, or engage in some interesting chatter. Feel free to ask me anything!"),
 HumanMessage(content='你是谁'),
 AIMessage(content='我是一个人工智能助手，被设计来帮助用户解答问题、提供信息或进行友好的对话。你可以把我想象成一个知识丰富的聊天机器人，随时准备与你交流。')]

In [35]:
from langchain.schema import messages_from_dict, messages_to_dict
import json
dicts = messages_to_dict(conversation.memory.chat_memory.messages)
dicts

[{'type': 'human',
  'data': {'content': 'Hi there!',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': "Hello! How can I assist you today? I'm here to provide information, answer questions, or engage in some interesting chatter. Feel free to ask me anything!",
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False,
   'tool_calls': [],
   'invalid_tool_calls': [],
   'usage_metadata': None}},
 {'type': 'human',
  'data': {'content': '你是谁',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': '我是一个人工智能助手，被设计来帮助用户解答问题、提供信息或进行友好的对话。你可以把我想象成一个知识丰富的聊天机器人，随时准备与你交流。',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False,


In [36]:
import pickle

f = open("./memory", 'wb')
pickle.dump(dicts,f)
f.close()

In [37]:
dicts_load = pickle.load(open("./memory", "rb"))
dicts_load

[{'type': 'human',
  'data': {'content': 'Hi there!',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': "Hello! How can I assist you today? I'm here to provide information, answer questions, or engage in some interesting chatter. Feel free to ask me anything!",
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False,
   'tool_calls': [],
   'invalid_tool_calls': [],
   'usage_metadata': None}},
 {'type': 'human',
  'data': {'content': '你是谁',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': '我是一个人工智能助手，被设计来帮助用户解答问题、提供信息或进行友好的对话。你可以把我想象成一个知识丰富的聊天机器人，随时准备与你交流。',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False,


In [39]:
# 向对话添加记忆
new_messages = messages_from_dict(dicts_load)
retrieved_chat_history = ChatMessageHistory(messages=new_messages)
retrieved_memory = ConversationBufferMemory(chat_memory=retrieved_chat_history)

conversation_reload = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=retrieved_memory
)
conversation_reload.predict(input="我回来了")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? I'm here to provide information, answer questions, or engage in some interesting chatter. Feel free to ask me anything!
Human: 你是谁
AI: 我是一个人工智能助手，被设计来帮助用户解答问题、提供信息或进行友好的对话。你可以把我想象成一个知识丰富的聊天机器人，随时准备与你交流。
Human: 我回来了
AI:
欢迎回来！有什么新话题或问题想要讨论吗？或者你需要我帮助解答上次我们谈话时留下的任何疑问吗？
> Finished chain.


'欢迎回来！有什么新话题或问题想要讨论吗？或者你需要我帮助解答上次我们谈话时留下的任何疑问吗？'

# 检索增强 与 文本向量器

## 文档导入

In [42]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_community.llms import Tongyi

model_ty = Tongyi(temperature=0.1)

In [9]:
# 不用print函数，直接用变量名称打印输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [46]:
from langchain.document_loaders import PyPDFLoader

loader_pdf = PyPDFLoader(r"D:\workspace\收发文件\新型冠状病毒肺炎防控方案（第八版）.pdf")
docs_pdf = loader_pdf.load()
type(docs_pdf)
len(docs_pdf)
type(docs_pdf[0])

list

129

langchain_core.documents.base.Document

In [48]:
docs_pdf[0].metadata
docs_pdf[0].page_content[0:100]

{'source': 'D:\\workspace\\收发文件\\新型冠状病毒肺炎防控方案（第八版）.pdf', 'page': 0}

'—2—新型冠状病毒肺炎防控方案\n（第八版）\n为指导各地做好新型冠状病毒肺炎（以下简称新冠肺炎，\nCOVID-19）疫情的常态化防控工作，全面落实“外防输入、内\n防反弹”的防控策略，根据新冠肺炎乙类传染'

In [50]:
from langchain.document_loaders import TextLoader
loader_txt = TextLoader(r'D:\workspace\收发文件\李卅移交资料.txt', encoding='utf8')
docs_txt = loader_txt.load()
len(docs_txt)

docs_txt[0].metadata
docs_txt[0].page_content[0:100]

1

{'source': 'D:\\workspace\\收发文件\\李卅移交资料.txt'}

'李卅移交资料：在itdevc07-EKB开发机上（10.66.58.121，用户名：itdevc07\\lcadmin，密码：2020@ekb），\n目录地址：D:\\DE-IoT 柴油机物联网.7z.00'

In [51]:
from langchain_community.document_loaders import WebBaseLoader
WEB_URL = "https://news.ifeng.com/c/8Y3TlIcTsj0"
loader_html = WebBaseLoader(WEB_URL)
docs_html = loader_html.load()

len(docs_html)
docs_html[0].metadata
docs_html[0].page_content[:100]

USER_AGENT environment variable not set, consider setting it to identify your requests.


1

{'source': 'https://news.ifeng.com/c/8Y3TlIcTsj0',
 'title': '足协原副主席王登峰被判17年_凤凰网',
 'description': '足协原副主席王登峰被判17年',
 'language': 'zh'}

'足协原副主席王登峰被判17年_凤凰网\n\n\n\n首页资讯视频直播凤凰卫视财经娱乐体育时尚汽车房产科技读书文化历史军事旅游佛教更多国学数码健康家居公益教育酒业美食资讯 > 大陆 > 正文足协原副主席王登峰被'

In [52]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 导入递归字符文本分割器
text_splitter_txt = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
# 导入文本
documents_txt = text_splitter_txt.split_documents(docs_txt)
len(documents_txt)

' \n* RecursiveCharacterTextSplitter 递归字符文本分割\nRecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，\n    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置\nRecursiveCharacterTextSplitter需要关注的是4个参数：\n\n* separators - 分隔符字符串数组\n* chunk_size - 每个文档的字符数量限制\n* chunk_overlap - 两份文档重叠区域的长度\n* length_function - 长度计算函数\n'

1

In [53]:
documents_txt[0].page_content

'李卅移交资料：在itdevc07-EKB开发机上（10.66.58.121，用户名：itdevc07\\lcadmin，密码：2020@ekb），\n目录地址：D:\\DE-IoT 柴油机物联网.7z.001~008。8个压缩文件用7-zip解压软件任意解压一个即可。\n主要资料位于解压后的 ./clickhouse 和./DMP开发 两个目录中。'

___

In [66]:
from sentence_transformers import SentenceTransformer

sentence_transformer  = SentenceTransformer('D:\workspace\project\LLM\models\m3e-base')
sentences = [
    '* Moka 此文本嵌入模型由 MokaAI 训练并开源，训练脚本使用 uniem',
    '* Massive 此文本嵌入模型通过**千万级**的中文句对数据集进行训练',
    '* Mixed 此文本嵌入模型支持中英双语的同质文本相似度计算，异质文本检索等功能，未来还会支持代码检索，ALL in one'
]

#Sentences are encoded by calling model.encode()
emb = sentence_transformer.encode(sentences)

In [61]:
query1 = "狗"
query2 = "猫"
query3 = "雨"
# 通过对应的 embedding 类生成 query 的 embedding。
emb1 = sentence_transformer.encode(query1)
emb2 = sentence_transformer.encode(query2)
emb3 = sentence_transformer.encode(query3)

In [63]:
import numpy as np
# 将返回结果转成 numpy 的格式，便于后续计算
emb1 = np.array(emb1)
emb2 = np.array(emb2)
emb3 = np.array(emb3)

np.dot(emb1, emb2)
np.dot(emb3, emb2)
np.dot(emb1, emb3)

320.04456

302.9192

300.03458

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
# sentenceTransformer = SentenceTransformer('D:\workspace\project\LLM\models\m3e-base')
embedding = SentenceTransformerEmbeddings(model_name='D:\workspace\project\LLM\models\m3e-base')


D:\Program\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
D:\Program\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [1]:
from langchain_community.document_loaders import WebBaseLoader
WEB_URL = "https://news.ifeng.com/c/8Y3TlIcTsj0"
loader_html = WebBaseLoader(WEB_URL)
docs_html = loader_html.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
docs_html

[Document(page_content='足协原副主席王登峰被判17年_凤凰网\n\n\n\n首页资讯视频直播凤凰卫视财经娱乐体育时尚汽车房产科技读书文化历史军事旅游佛教更多国学数码健康家居公益教育酒业美食资讯 > 大陆 > 正文足协原副主席王登峰被判17年山东高法2024年03月18日 16:55:00 2024年3月18日上午，山东省德州市中级人民法院一审公开宣判教育部体育卫生与艺术教育司原司长王登峰贪污、受贿案，以贪污罪判处其有期徒刑十四年，并处罚金人民币四百万元；以受贿罪判处其有期徒刑十一年，并处罚金人民币一百万元，决定执行有期徒刑十七年，并处罚金人民币五百万元。查扣在案的贪污赃款依法返还被害单位，受贿犯罪违法所得及孳息依法没收，上缴国库。经审理查明：一、贪污罪。2016年6月至2022年8月，被告人王登峰利用担任教育部体育卫生与艺术教育司司长等的职务便利，单独或伙同他人，采取侵吞、骗取等手段，非法占有公共财物共计4670.13万余元，其中180.49万余元未遂。二、受贿罪。2006年1月至2022年春节前，被告人王登峰利用担任教育部体育卫生与艺术教育司司长等职务便利，为他人在项目承揽、赛事承办等方面谋取利益，单独或伙同他人收受相关单位或个人给予的财物，共计折合人民币966.469万元。德州市中级人民法院认为，被告人王登峰身为国家工作人员，利用职务上的便利，侵吞、骗取公共财物，数额特别巨大，其行为构成贪污罪；身为国家工作人员，利用职务上的便利，为他人谋取利益，非法收受他人财物，数额特别巨大，其行为构成受贿罪。王登峰犯数罪，应予并罚。王登峰具有坦白情节，积极退缴全部赃款赃物及孳息，部分贪污犯罪系未遂，具有法定、酌定从轻处罚情节，依法可对其从轻处罚。法院遂依法作出上述判决。王登峰个人履历：1981.09-1985.07北京大学心理学系本科学习，获心理学学士学位；1994.07-1996.12北京大学团委书记（1995.08破格晋升教授）；1996.12-1997.07北京大学党委常委、校团委书记；1999.02-2006.04北京大学党委副书记（2004.02-2006.01挂职江西省吉安市委副书记），北京大学人格与社会心理学研究中心主任（2003.11成立并兼）；2006.04-国家语言文字工作委员会副主任、教育部语言文字应用管理司司长；2011年11月

In [7]:
# persist_directory允许我们将目录保存到磁盘上
# 注意from_documents每次运行都是把数据添加进去
vectordb = Chroma.from_documents(documents=docs_html, embedding=embedding, persist_directory="D:\workspace\project\LLM\Chroma" )

In [8]:
vectordb.persist()

D:\Program\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [3]:
vectordb_load = Chroma(
    persist_directory="D:\workspace\project\LLM\Chroma",
    embedding_function=embedding
)

In [4]:
vectordb_load._collection.count()

1

In [5]:
vectordb_load.similarity_search("足协")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


[Document(page_content='足协原副主席王登峰被判17年_凤凰网\n\n\n\n首页资讯视频直播凤凰卫视财经娱乐体育时尚汽车房产科技读书文化历史军事旅游佛教更多国学数码健康家居公益教育酒业美食资讯 > 大陆 > 正文足协原副主席王登峰被判17年山东高法2024年03月18日 16:55:00 2024年3月18日上午，山东省德州市中级人民法院一审公开宣判教育部体育卫生与艺术教育司原司长王登峰贪污、受贿案，以贪污罪判处其有期徒刑十四年，并处罚金人民币四百万元；以受贿罪判处其有期徒刑十一年，并处罚金人民币一百万元，决定执行有期徒刑十七年，并处罚金人民币五百万元。查扣在案的贪污赃款依法返还被害单位，受贿犯罪违法所得及孳息依法没收，上缴国库。经审理查明：一、贪污罪。2016年6月至2022年8月，被告人王登峰利用担任教育部体育卫生与艺术教育司司长等的职务便利，单独或伙同他人，采取侵吞、骗取等手段，非法占有公共财物共计4670.13万余元，其中180.49万余元未遂。二、受贿罪。2006年1月至2022年春节前，被告人王登峰利用担任教育部体育卫生与艺术教育司司长等职务便利，为他人在项目承揽、赛事承办等方面谋取利益，单独或伙同他人收受相关单位或个人给予的财物，共计折合人民币966.469万元。德州市中级人民法院认为，被告人王登峰身为国家工作人员，利用职务上的便利，侵吞、骗取公共财物，数额特别巨大，其行为构成贪污罪；身为国家工作人员，利用职务上的便利，为他人谋取利益，非法收受他人财物，数额特别巨大，其行为构成受贿罪。王登峰犯数罪，应予并罚。王登峰具有坦白情节，积极退缴全部赃款赃物及孳息，部分贪污犯罪系未遂，具有法定、酌定从轻处罚情节，依法可对其从轻处罚。法院遂依法作出上述判决。王登峰个人履历：1981.09-1985.07北京大学心理学系本科学习，获心理学学士学位；1994.07-1996.12北京大学团委书记（1995.08破格晋升教授）；1996.12-1997.07北京大学党委常委、校团委书记；1999.02-2006.04北京大学党委副书记（2004.02-2006.01挂职江西省吉安市委副书记），北京大学人格与社会心理学研究中心主任（2003.11成立并兼）；2006.04-国家语言文字工作委员会副主任、教育部语言文字应用管理司司长；2011年11月

#### 检索式问答链

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts.chat import ChatPromptTemplate
from langchain_community.llms import Tongyi

# 创建提示词模板
prompt = ChatPromptTemplate.from_template("""使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话，你的回答中不可以带有星号。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语。
        请注意！回答语言与提问语言保持一致。
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
<context>
{context}
</context>

Question: {input}""")
#创建检索链
llm = Tongyi(temperature=0.1)
document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vectordb_load.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
response = retrieval_chain.invoke({
    "input": "王登峰是谁?"
})
response["answer"]

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'王登峰曾任教育部体育卫生与艺术教育司司长、国家语言文字工作委员会副主任、足协原副主席等职务。感谢你的提问。'

In [12]:
# 比较：原生模型调用与retrieval_chain调用的区别：
llm.invoke("王登峰是谁?")

'王登峰，男，汉族，1963年10月生，山西孝义人，1984年12月加入中国共产党，1987年7月参加工作，北京大学哲学系哲学专业毕业，研究生学历，教育学博士学位，教授。\n\n现任教育部体育卫生与艺术教育司司长，中国大学生体育协会主席，中国中学生体育协会主席。'

#### 检索式对话模型

In [24]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

ValidationError: 1 validation error for ConversationSummaryMemory
llm
  Can't instantiate abstract class BaseLanguageModel with abstract methods agenerate_prompt, apredict, apredict_messages, generate_prompt, invoke, predict, predict_messages (type=type_error)

In [25]:
res = qa.invoke(
    {"question": "王登峰是谁?"}
)
res["answer"]

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


' Wang Dengfeng is a former vice chairman of the Chinese Football Association and former head of the Department of Physical Education, Health, and Arts Education at the Ministry of Education in China. He has been sentenced to 17 years in prison for corruption charges, including贪污 and受贿.'

In [16]:
res = qa.invoke(
    {"question": "王登峰怎么样了?"}
)
res["answer"]

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


" Wang Dengfeng, former vice chairman of the Football Association, was sentenced to 17 years in prison by the Dezhou Intermediate People's Court in Shandong Province for贪污 and受贿 crimes."

In [26]:
memory.chat_memory.messages

[HumanMessage(content='王登峰是谁?'),
 AIMessage(content=' Wang Dengfeng is a former vice chairman of the Chinese Football Association and former head of the Department of Physical Education, Health, and Arts Education at the Ministry of Education in China. He has been sentenced to 17 years in prison for corruption charges, including贪污 and受贿.')]

In [22]:
memory.chat_memory.clear()